In [1]:
from __future__ import  print_function
import torch
from torch import nn, optim
from torch.nn import functional as F
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import os
from torch.autograd import Variable
from torch.utils.data import DataLoader, Dataset
from torch.utils.data.sampler import SubsetRandomSampler
import torchvision
from torchvision import transforms
import cv2
from torchvision.utils import save_image

In [2]:
class MyDataset(Dataset):
    def __init__(self, data_npz):
        x_npz = np.load(data_npz)
        x_ww = x_npz['arr_0']
        self.len = x_ww.shape[0]
        self.dim = x_ww.shape[1]
        self.data = torch.from_numpy(x_ww)
    
    def __getitem__(self,index):
        return self.data[index]
            
    def __len__(self):
        return self.len

In [3]:
datset = MyDataset('../x_ww_bw_50176_pre-processed.npz')

In [4]:
validation_split = .2
shuffle_dataset = True
random_seed = 42

In [5]:
dataset_size = len(datset)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))
if shuffle_dataset :
    np.random.seed(random_seed)
    np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

In [6]:
train_sampler = SubsetRandomSampler(train_indices)
val_sampler = SubsetRandomSampler(val_indices)

In [7]:
train_loader = DataLoader(dataset=datset, batch_size=32, sampler=train_sampler, num_workers=2)

In [8]:
test_loader = DataLoader(dataset=datset, batch_size=32, sampler=val_sampler, num_workers=2)

In [9]:
device = torch.device('cuda')

In [10]:
class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()
        
        self.fc1 = nn.Linear(50176, 400)
        self.fc21 = nn.Linear(400, 20)
        self.fc22 = nn.Linear(400, 20)
        self.fc3 = nn.Linear(20, 400)
        self.fc4 = nn.Linear(400, 50176)
        
    def encode(self, x):
        h1 = F.relu(self.fc1(x))
        return self.fc21(h1), self.fc22(h1)
    
    def reparametrize(self, mu, logvar):
        std = torch.exp(0.5*logvar)
        eps = torch.rand_like(std)
        return eps.mul(std).add_(mu)
    
    def decode(self, z):
        h3 = F.relu(self.fc3(z))
        return torch.sigmoid(self.fc4(h3))
    
    def forward(self, x):
        mu, logvar = self.encode(x.view(-1,50176))
        z = self.reparametrize(mu, logvar)
        return self.decode(z), mu, logvar

In [11]:
model = VAE().to(device)

In [12]:
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [13]:
def loss_func(recon_x, x, mu, logvar):
    BCE = F.binary_cross_entropy(recon_x, x.view(-1, 50176), reduction='sum')
    
    KLD = -0.5* torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    
    return BCE + KLD

In [14]:
def train(epoch):
    model.train()
    train_loss = 0
    for batch_idx, data in enumerate(train_loader, 0):
        data = data.to(device)
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(data)
        loss = loss_func(recon_batch, data, mu, logvar)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        if batch_idx % 100 == 0:
            print('train epoch: {} [{}/{} ({:.2f}%)]\tLoss:{:.3f}'.format(
                    epoch, batch_idx*len(data), len(train_loader.dataset),
                    100.* batch_idx/len(train_loader), loss.item()/len(data)))
            
#        print('=====> Epoch:{} Avarage Loss: {:.4f}'.format(
#                epoch, train_loss/ len(train_loader.dataset)))

def test(epoch):
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for batch_idx, data in enumerate(train_loader, 0):
            data = data.to(device)
            recon_batch, mu, logvar = model(data)
            test_loss += loss_func(recon_batch, data, mu, logvar).item()
            if batch_idx ==0:
                n = min(data.size(0), 8)
                comparison = torch.cat([data.view(32,1,224,224)[:n], recon_batch.view(32,1,224,224)[:n]])
                save_image(comparison.cpu(), 'results/reconstruction_' + str(epoch) + '.png', nrow=n)
                
    test_loss /= len(test_loader.dataset)
    print('=====> test set loss:{:.4f}'.format(test_loss))
            

In [16]:
for epoch in range(1,10):
    train(epoch)
    test(epoch)
    with torch.no_grad():
        sample = torch.randn(32, 20).to(device)
        sample = model.decode(sample).cpu()
        save_image(sample.view(32,1,224,224), 'results/sample_' + str(epoch) + '.png')

train epoch: 1 [0/49993 (0.00%)]	Loss:34917.445
train epoch: 1 [3200/49993 (8.00%)]	Loss:11030.261
train epoch: 1 [6400/49993 (16.00%)]	Loss:9162.209
train epoch: 1 [9600/49993 (24.00%)]	Loss:9211.058
train epoch: 1 [12800/49993 (32.00%)]	Loss:8712.184
train epoch: 1 [16000/49993 (40.00%)]	Loss:8615.823
train epoch: 1 [19200/49993 (48.00%)]	Loss:8249.754
train epoch: 1 [22400/49993 (56.00%)]	Loss:9466.723
train epoch: 1 [25600/49993 (64.00%)]	Loss:8768.545
train epoch: 1 [28800/49993 (72.00%)]	Loss:8393.656
train epoch: 1 [32000/49993 (80.00%)]	Loss:8744.381
train epoch: 1 [35200/49993 (88.00%)]	Loss:9029.450
train epoch: 1 [38400/49993 (96.00%)]	Loss:8979.657
=====> test set loss:6675.1595
train epoch: 2 [0/49993 (0.00%)]	Loss:8066.669
train epoch: 2 [3200/49993 (8.00%)]	Loss:8164.577
train epoch: 2 [6400/49993 (16.00%)]	Loss:8207.202
train epoch: 2 [9600/49993 (24.00%)]	Loss:7682.562
train epoch: 2 [12800/49993 (32.00%)]	Loss:7999.230
train epoch: 2 [16000/49993 (40.00%)]	Loss:8166.0